In [1]:
import os
import openai
from langchain_openai import ChatOpenAI
from langchain_deepseek import ChatDeepSeek
from langchain.prompts import PromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import random
import pandas as pd
import re
from langchain_core.output_parsers import PydanticOutputParser
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

True

In [2]:
# llm = ChatOpenAI(model="o3-mini")
llm = ChatDeepSeek(
    model="deepseek-reasoner"
)

In [3]:
master_prompt = """

You are a DeFi Agent Launcher for Xade AI: A meta-agent that creates agents for users that can fetch, analyze DeFi data as well as execute actions based on them for the data. 

The input you will receive will be a complex task that is phrased in a very elementary manner, along with a few customisations that the user may want. However, it may have several missing information which you will need to fill in using your own judgement.
Your task is to review the challenge, instruct and allocate tasks to your underlying LLM agents to obtain the relevant data, synthesize it, customize responses and execute some actions.

---

You will have access to the following LLM agents:
- A DeFi Data Fetcher: An agent that can fetch DeFi data from various sources, and then write functions to create information from the given data. 
This agent will have access to market and social metrics for DeFi tokens and categories. Additionally, it can also create technical indicators and other metrics based on the data.
It will use a RAG model to fetch relevant data sources based on the input you provide for the RAG model. 
RAG Model:
  You must craft the input for the RAG model keepping in mind that it can only retrieve functions to get the data. 
  If you need to identify any specific tokens on a specific blockchain or from a specific sector of DeFi (for e.g. ReFi), ask the RAG model for the functions to get those tokens.
  Directly mention the data points (price7d, altRank, galaxyScore) you need and/or the kind of data you need (sentiment-related, volatility-related) or any category-specific data.
  Currently, the RAG model has access to Mobula (for market data) and LunarCrush (for social data).
  Any filters you want for the data from the RAG model should be mentioned in the instructions for the DeFi Fetcher, and not in the RAG model input
The Data Fetcher will then create information based off of the data derived from the RAG model, while processing it as well. 

- A DeFi Analyzer: An agent that can analyze DeFi data and provide insights based on the data. 
This agent generates a data-driven analysis and evaluation of the data while using suitable methodologies (STARE, CAR , SOAR, IDEA frameworks). 
It is smart enough to understand the context of the data and provide insights based on the data.
Based on your instructions and the data given by the Data Fetcher, it can model its analysis to:
  - Discover emerging trends
  - Analyze technical indicators such as RSI, MACD, and Bollinger Bands to predict movements
  - Interpret social media metrics to gauge market sentiment and mindshare
Thus, ensure you give instructions such that the analyzer can provide insights relevant to the user.


- A DeFi Executor: An agent that can execute actions based on the data fetched and analyzed by the other two agents. 
This agent can devise trading strategies, execute trades, post on X and the XADE Terminal every few minutes, reply to various pre-defined indiviuals, join Twitter spaces, perform portoflio analysis, and other such actions.
It's execution will be performed by calling the relevant functions.
It will execute the actions you ask it to, based on the data fetched and analyzed by the other two agents.
It will have access to a memory of it's tasks, so it can remember its previous context and actions and adjust future actions accordingly.
Here is a list of it's abilities:
  - Post on X and the XADE Terminal periodically
  - Reply to posts by various pre-defined individuals
  - Join Twitter spaces and engage in discussions
  - Perform portfolio analysis for a given wallet address
  - Devise trading strategies based on the user's priorities and the analyzed data.
  - For text-based actions, it can customize responses based on the user's preferences.

---

When creating the instructions for the LLMs to execute, break your instructions into a logical, step-by-step order, using the specified format:
    - Primary actions must be clearly defined. You must define the end-goal that the LLM is trying to achieve through its actions.
    - Sub-actions must be indented and clearly defined, beginning on a new line.
    - Specify conditions using clear 'if...then...else' statements
    - Ensure that there is no ambiguity in any part of your plan. You must be specific and clear about what to use and how.
    - The instructions generated must be extremely detailed and thorough with explanations at every step. They must first outline the primary instruction for the LLM and then provide direct and thorough explanations
    - Do not use any markdown formatting.
    
---

TASK:
{input}

"""

In [4]:
from typing import Optional

from pydantic import BaseModel, Field

class ElPlan(BaseModel):
    data_agent_instructions: str = Field(description="The task for the DeFi Data Fetcher and the insturctions for execution.")
    data_rag_prompt: str = Field(description="The prompt for the RAG model to fetch relevant data sources")
    analyzer: str = Field(description="The task for the analyzer and the instructions for execution (detailed). This task must be possible to fulfill with data retreived by the Data Fetcher.")
    executor: str = Field(description="The task for the executor and the instructions to fulfill it. All specific instructions for each action can be included here as well. All miscellaneous instructions can be included here.")

parser = PydanticOutputParser(pydantic_object=ElPlan)

In [5]:
master_prompt = PromptTemplate(
    template=master_prompt+"\n{format_instructions}\nInstructions:\n",
    input_variables=["input"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [6]:
input = """
Agent Name & Description:
Agent ETH 


Query:
Analysis of trending projects on Ethereum

Actions:
  Sentient Posting:
  - Posting on X and XADE Terminal 
  - Every 5 minutes
  - Reply on tweets by Vitalik Buterin (@VitalikButerin)
  - Reply to mentions and quotes of oneself (@AgentETH)

Specific Data Sources to Use:
- Market Data
- Social Data
- News Feeds

Style:
Answer like a Degen Analyst

Examples:


INSTRUCTIONS:
"""

response = llm.invoke(master_prompt.format(input=input))

In [7]:
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
import json

def display_agent_output(response_content):
    console = Console()
    response_content = response_content.strip('```json\n').strip('\n```')
    try:
        data = json.loads(response_content)
    except:
        console.print("[bold red]Error parsing response as JSON[/bold red]")
        return
    
    table = Table(show_header=True, header_style="bold")
    table.add_column("Component", style="italic white")
    table.add_column("Instructions", style="")
    
    for key, value in data.items():
        formatted_key = key.replace('_', ' ').title()
        table.add_row(formatted_key, value)
    
    console.print(Panel(
        table,
        title="[bold yellow]🤖 DeFi Agent Instructions[/bold yellow]",
        border_style="yellow"
    ))

In [8]:
display_agent_output(response.content)

╭────────────────────────────────────────── 🤖 DeFi Agent Instructions ───────────────────────────────────────────╮
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │
│ ┃ Component               ┃ Instructions                                                                      ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ │
│ │ Data Agent Instructions │ Fetch and process data on trending Ethereum projects using the following steps:   │ │
│ │                         │ 1. Collect market data (price7d, 24h volume, altRank, volatility) and social data │ │
│ │                         │ (galaxyScore, social engagements, sentiment) for top 10 trending Ethereum         │ │
│ │                         │ projects.                                                                         │ │
│ │                         │ 2. Filter projects with >15% price7d change, >$1M 24h volume, and galaxyScore >60 │ │
│ │                         │ to ensure relevance.                                                              │ │
│ │                         │ 3. Calculate 24h volatility using standard deviation of hourly prices and add as  │ │
│ │                         │ custom metric.                                                                    │ │
│ │                         │ 4. Cross-reference with news feeds for project-specific announcements or          │ │
│ │                         │ partnerships.                                                                     │ │
│ │                         │ 5. Compile data into a ranked list combining altRank (30% weight), galaxyScore    │ │
│ │                         │ (30%), volatility-adjusted volume (20%), and news mentions (20%).                 │ │
│ │ Data Rag Prompt         │ Retrieve functions to get:                                                        │ │
│ │                         │ 1. Top 10 Ethereum projects by LunarCrush AltRank from last 24h                   │ │
│ │                         │ 2. Mobula market data (price7d, 24h volume, volatility) for these projects        │ │
│ │                         │ 3. Social metrics (galaxyScore, social engagements, sentiment polarity) from      │ │
│ │                         │ LunarCrush                                                                        │ │
│ │                         │ 4. Recent news headlines (last 6h) containing project names from CryptoPanic API  │ │
│ │ Analyzer                │ Execute IDEA framework analysis:                                                  │ │
│ │                         │ 1. Identify Patterns: Compare volatility spikes with social engagement surges     │ │
│ │                         │ 2. Diagnose Causes: Cross-reference price movements with news events and          │ │
│ │                         │ technical indicators (RSI, MACD crossovers)                                       │ │
│ │                         │ 3. Evaluate Trends: Calculate 4h moving average slope vs. social sentiment        │ │
│ │                         │ trajectory                                                                        │ │
│ │                         │ 4. Act on Insights: Highlight 3 projects with:                                    │ │
│ │                         │    - Bullish divergence (price down but social up)                                │ │
│ │                         │    - News catalysts + above-average volume                                        │ │
│ │                         │    - Oversold RSI (<30) with increasing whale transactions                        │ │
│ │                         │ Format analysis with degenspeak: 'GM fren! Spotted a V-shaped recovery brewing    │ │
│ │                         │ for $PROJECT - RSI looking oversold while whales accumulating. Could send it to   │ │
│ │                         │ Valhalla if the devs drop t

In [4]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import inspect

# ----------------------------
# Assume KNOWLEDGE_BASE is built from our 20 functions.
# For demonstration, here's a dummy KNOWLEDGE_BASE for two functions.
# In your system, KNOWLEDGE_BASE will contain all 20 functions.
# For example:
# KNOWLEDGE_BASE = {
#    "calculateSMA": {"description": <docstring>, "code": <source code>},
#    ... 
# }

# For the purpose of this demo, let's assume KNOWLEDGE_BASE has been built already.
# Uncomment the following line if you already built it:
# KNOWLEDGE_BASE = build_knowledge_base()

# In our test, we simulate KNOWLEDGE_BASE with the functions defined previously.
# (You should replace this with your actual KNOWLEDGE_BASE dictionary from the previous step.)
def calculateSMA(price_history_output: list, period: int) -> float:
    """
    Function Name: calculateSMA
    Description: Computes the Simple Moving Average (SMA) using historical price data.
    Inputs:
        - price_history_output: List of historical prices.
        - period: Number of data points to average.
    Processing:
        - Extract the last 'period' prices.
        - Compute SMA = sum(last_period_prices) / period.
    Output:
        - A float representing the SMA.
    """
    if len(price_history_output) < period:
        raise ValueError("Not enough data points to compute SMA.")
    return sum(price_history_output[-period:]) / period

def calculateEMA(price_history_output: list, period: int) -> float:
    """
    Function Name: calculateEMA
    Description: Computes the Exponential Moving Average (EMA) with more weight on recent prices.
    Inputs:
        - price_history_output: List of historical prices.
        - period: The EMA period.
    Processing:
        - Initialize EMA using the SMA of the first 'period' data points.
        - For each subsequent price, update EMA with smoothing factor k = 2/(period+1).
    Output:
        - A float representing the final EMA value.
    """
    if len(price_history_output) < period:
        raise ValueError("Not enough data points to compute EMA.")
    ema = sum(price_history_output[:period]) / period
    k = 2 / (period + 1)
    for price in price_history_output[period:]:
        ema = price * k + ema * (1 - k)
    return ema

# Build a simulated knowledge base using these two functions (extend this to all 20)
def build_knowledge_base():
    functions = [calculateSMA, calculateEMA]  # Replace with all 20 functions in practice.
    kb = {}
    for func in functions:
        source = inspect.getsource(func)
        doc = func.__doc__
        kb[func.__name__] = {"description": doc, "code": source}
    return kb

KNOWLEDGE_BASE = build_knowledge_base()


In [5]:
# ----------------------------
# Build the Vector Database using FAISS and SentenceTransformer
# ----------------------------

# Prepare the list of function names and corresponding descriptions from the knowledge base.
function_names = list(KNOWLEDGE_BASE.keys())
function_descriptions = [KNOWLEDGE_BASE[name]["description"] for name in function_names]





In [6]:
len(function_descriptions)

2

In [7]:
# Initialize the embedding model (using a lightweight model here; adjust as needed).
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")



In [8]:
# Compute embeddings for each function description.
embeddings = embedding_model.encode(function_descriptions)
embeddings = np.array(embeddings).astype("float32")

In [9]:

dim = embeddings.shape[1]
faiss_index = faiss.IndexFlatL2(dim)
faiss_index.add(embeddings)

In [ ]:

# ----------------------------
# RAG Search Function Using the Vector Database
# ----------------------------

def rag_search_vector(query: str, top_k: int = 3) -> dict:
    """
    RAG Search with a vector database.
    Given a query string, this function computes its embedding and retrieves the top_k
    function entries whose description embeddings are closest.
    
    Inputs:
        - query: A string query.
        - top_k: Number of top results to return.
    Output:
        - A dictionary mapping function names to their 'description' and 'code'.
    """
    query_embedding = embedding_model.encode([query])
    query_embedding = np.array(query_embedding).astype("float32")
    
    distances, indices = faiss_index.search(query_embedding, top_k)
    results = {}
    for idx in indices[0]:
        func_name = function_names[idx]
        results[func_name] = KNOWLEDGE_BASE[func_name]
    return results


In [11]:
sample_query = "Retrieve functions to get the Simple Moving Average (SMA) of a price history."

matching_functions = rag_search_vector(sample_query, top_k=3)

if matching_functions:
  print("Matching Functions for query:", sample_query)
  for name, details in matching_functions.items():
      print(f"\nFunction: {name}\nDescription:\n{details['description']}\nCode:\n{details['code']}\n{'-'*40}")
else:
  print("No matching functions found.")

Matching Functions for query: Retrieve functions to get the Simple Moving Average (SMA) of a price history.

Function: calculateSMA
Description:

    Function Name: calculateSMA
    Description: Computes the Simple Moving Average (SMA) using historical price data.
    Inputs:
        - price_history_output: List of historical prices.
        - period: Number of data points to average.
    Processing:
        - Extract the last 'period' prices.
        - Compute SMA = sum(last_period_prices) / period.
    Output:
        - A float representing the SMA.
    
Code:
def calculateSMA(price_history_output: list, period: int) -> float:
    """
    Function Name: calculateSMA
    Description: Computes the Simple Moving Average (SMA) using historical price data.
    Inputs:
        - price_history_output: List of historical prices.
        - period: Number of data points to average.
    Processing:
        - Extract the last 'period' prices.
        - Compute SMA = sum(last_period_prices) / per